In [ ]:
!pip install python-terrier



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 73.2 MB/s 
     |████████████████████████████████| 69 kB 8.4 MB/s 
     |████████████████████████████████| 303 kB 66.0 MB/s 
     |████████████████████████████████| 46 kB 5.0 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 126 kB 69.8 MB/s 
     |████████████████████████████████| 1.2 MB 85.1 MB/s 
     |████████████████████████████████| 72 kB 1.9 MB/s 
     |████████████████████████████████| 294 kB 99.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 285 kB 69.2 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=7ff09fca25020155bb3dd39a2bf0282c6a5e04085e562

In [ ]:
import pyterrier as pt
pt.init()


terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
mod_queries = np.load('/content/drive/MyDrive/IRDM/queries_doc_test3.npy',allow_pickle='TRUE').item()

In [ ]:

def mergmast(list_of_dicts):
    master={}
    for i in range(len(list_of_dicts)):
      open = np.load(list_of_dicts[i],allow_pickle='TRUE').item()
      master.update(open)
    return master

In [ ]:
import numpy as np 
mod_docs = np.load('/content/drive/MyDrive/IRDM/master_set_docs.npy',allow_pickle='TRUE').item()

In [ ]:
def make_experiment_file(dict_input):
  output_file={}
  for keys in dict_input.keys():
    for k in range(len(dict_input[keys])):
      if keys not in output_file.keys():
        output_file.update({keys:[dict_input[keys][k][0]]})
      else:
        output_file[keys].append(dict_input[keys][k][0])
  return output_file

In [ ]:
queries_info= make_experiment_file(mod_queries)

In [ ]:
documents = make_experiment_file(mod_docs)

In [ ]:
docs = list(documents.keys())

In [ ]:
len(docs)

149802

In [ ]:


def fill_tokens(small_chunks):
  return'wiki_'+('_').join(small_chunks.split(' '))

def wiki_tokens_to_wikified_text(input_list):
  output_list=[]
  for i in input_list:
    output_list.append(fill_tokens(i))
  return output_list

def transform_dict(input_dict):
  new_dict={}
  for keys in input_dict.keys():
    new_dict.update({keys: wiki_tokens_to_wikified_text(input_dict[keys])})
  return new_dict


In [ ]:
transformed_doc=transform_dict(documents)

In [ ]:
transformed_query=transform_dict(queries_info)

In [ ]:
np.save('/content/drive/MyDrive/IRDM/wiki_retrieval_ready_docs_test.npy',transformed_doc) 


In [ ]:
def return_only_files(segment_list):
  just_docs=[]
  for seg in range(len(segment_list)):
    just_docs.append(segment_list[seg].split('_')[0]+'.xml')
  
  #just_docs=list(set(just_docs))
  return just_docs

In [ ]:
def return_dict(segment_list):
  segment={}
  for seg in range(len(segment_list)):
    if segment_list[seg].split('_')[0] not in segment.keys():
      segment.update({segment_list[seg].split('_')[0]:1})
    else:
      segment[segment_list[seg].split('_')[0]]+=1
  return segment

In [ ]:
check=return_dict(docs)

In [ ]:
files= return_only_files(docs)

In [ ]:
import os
to_be_read=[]
for i in range(8):
  name_dir="/content/drive/MyDrive/IRDM/exp_data/xml_dir/"+str(i)
  for root, folder, data in os.walk(name_dir):
    for a in data:
      if ".xml" in a:
        if a in files:
          to_be_read.append((os.path.join(root,a)))

In [ ]:
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
qrels["qid"]=qrels["qid"].astype(int)
qrels["qid"]=qrels["qid"]-1
qrels["qid"]=qrels["qid"].astype("str")
qrels["file_name"]=qrels["docno"].apply(lambda x: x.split(":")[-1].split("_")[0]+".xml")
relevance_files=list(set(list(qrels["file_name"])))

In [ ]:
relevant=[]
for i in range(8):
  name_dir="/content/drive/MyDrive/IRDM/exp_data/xml_dir/"+str(i)
  for root, folder, data in os.walk(name_dir):
    for a in data:
        if a in relevance_files:
          relevant.append((os.path.join(root,a)))

In [ ]:
np.save('/content/drive/MyDrive/IRDM/files_rel.npy', master_dict) 

3421

In [ ]:
testing= np.load('/content/drive/MyDrive/IRDM/files_rel_train.npy',allow_pickle='TRUE').item()

In [ ]:
testing_2=np.load('/content/drive/MyDrive/IRDM/files_rel.npy',allow_pickle='TRUE').item()

In [ ]:

def mergmast(list_of_dicts):
    master={}
    for i in range(len(list_of_dicts)):
      open = np.load(list_of_dicts[i],allow_pickle='TRUE').item()
      master.update(open)
    return master

In [ ]:
total_stack=mergmast(["/content/drive/MyDrive/IRDM/files_rel_train.npy","/content/drive/MyDrive/IRDM/files_rel.npy"])

In [ ]:
len(transformed_doc.keys())

149802

In [ ]:
def reduce_d(master_list,append):
  new={}
  for k in append.keys():
    if k in master_list:
      new.update({k:append[k]})
  return new

Generate TREC XML files for Podcast SE (Can be used for Podcast Small as well)

In [ ]:
training=reduce_d(list(transformed_doc.keys()),total_stack)

In [ ]:
TEMPLATE = """<DOC>
<DOCNO>{}</DOCNO>
{}
</DOC>"""

In [ ]:
with open("/content/drive/MyDrive/IRDM/just_doc_file_test.xml", 'w') as ofile:
        for j in training.keys():
          show=j.split('_')[0]
          seg=j.split('_')[1]
          doc_id='{}_{}'.format(show,seg)
          doc_text=training[j].split('\n')[0]

          to_write = TEMPLATE.format(doc_id,doc_text)
          print(to_write, file=ofile)


In [ ]:
with open("/content/drive/MyDrive/IRDM/wikified_file_for_wiki_test.xml", 'w') as ofile:
        for j in training.keys():
          show=j.split('_')[0]
          seg=j.split('_')[1]
          doc_id='{}_{}'.format(show,seg)
          doc_text=training[j].split('\n')[0]+' '.join(transformed_doc[j])

          to_write = TEMPLATE.format(doc_id,doc_text)
          print(to_write, file=ofile)


In [ ]:
with open("/content/drive/MyDrive/IRDM/wikified_file_for_wiki_freq_2_test.xml", 'w') as ofile:
        for j in training.keys():
          show=j.split('_')[0]
          seg=j.split('_')[1]
          doc_id='{}_{}'.format(show,seg)
          doc_text=training[j].split('\n')[0]+' '.join(transformed_doc[j])+' '.join(transformed_doc[j])

          to_write = TEMPLATE.format(doc_id,doc_text)
          print(to_write, file=ofile)

In [ ]:
with open("/content/drive/MyDrive/IRDM/wikified_file_for_wiki_freq_5_test.xml", 'w') as ofile:
        for j in training.keys():
          show=j.split('_')[0]
          seg=j.split('_')[1]
          doc_id='{}_{}'.format(show,seg)
          doc_text=training[j].split('\n')[0]+' '.join(transformed_doc[j])*5

          to_write = TEMPLATE.format(doc_id,doc_text)
          print(to_write, file=ofile)

In [ ]:
import requests
import spacy
from collections import Counter
import matplotlib.pyplot as plt
import spacy

import argparse
import sys
import string
import spacy
import xml.etree.ElementTree as ET


Generate Wikified XML files 

In [ ]:
def update_xml(xml, nouns, f_name):
    n_queries = 0

    tree = ET.parse(xml)
    for node in tree.iter(): 
        if node.tag == 'query':
            n_queries += 1
            print(node.text)  
            gather = ' '.join(list(set(nouns[n_queries-1])))
            tmp = node.text + ' ' + gather
            print(tmp)
            tmp = tmp.lower()
            node.text = tmp
    if n_queries != len(nouns):
        print('Warning: n_queries do not match with number '
                'of descriptions parsed', file=sys.stderr)

    tree.write(f_name)


top="/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml"
nlp = spacy.load("en_core_web_sm")
tree = ET.parse(top)

wiki=[]
for node in tree.iter():
    if node.tag=='num':
        key=node.text
    if node.tag == 'query':
      if key in transformed_query.keys():
        wiki.append(transformed_query[key])
      else:
        wiki.append([])
update_xml(top, wiki, "/content/drive/MyDrive/IRDM/wikified_query_tests.xml")

trump call ukrainian president
trump call ukrainian president wiki_White_House wiki_Telecommunications wiki_2019_J2_League wiki_Trump_card_games wiki_President_of_the_United_States wiki_Ball_culture wiki_White_people wiki_Ukraine
boeing 737 crash causes
boeing 737 crash causes wiki_Hundred_county_division wiki_Boeing_737 wiki_Maxima_and_minima wiki_Activism wiki_Stock_market_crash wiki_2019_J2_League
how to cook turkey
how to cook turkey wiki_Guatemalan_Civil_War wiki_Indian_cuisine wiki_Turkey_bird
imran khan career
imran khan career wiki_Pakistan wiki_Pennsylvania27s_22nd_congressional_district wiki_Career wiki_Khan_title
drug addiction recovery
drug addiction recovery wiki_Addiction wiki_Drug_rehabilitation
near death experiences
near death experiences wiki_Experience wiki_Middle_East wiki_Death
podcast about podcasts
podcast about podcasts wiki_Podcast wiki_Aboutness
causes and prevention of wildfires
causes and prevention of wildfires wiki_Wildfire wiki_Attribution_of_recent_clima

In [ ]:
def update_xml(xml, nouns, f_name):
    n_queries = 0

    tree = ET.parse(xml)
    for node in tree.iter(): 
        if node.tag == 'query':
            n_queries += 1
            print(node.text)  
            gather = ' '.join(list(set(nouns[n_queries-1])))
            tmp = node.text + ' ' + gather
            print(tmp)
            tmp = tmp.lower()
            node.text = tmp
    if n_queries != len(nouns):
        print('Warning: n_queries do not match with number '
                'of descriptions parsed', file=sys.stderr)

    tree.write(f_name)


top="/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels"
nlp = spacy.load("en_core_web_sm")
tree = ET.parse(top)

wiki=[]
for node in tree.iter():
    if node.tag=='num':
        key=node.text
    if node.tag == 'query':
        wiki.append(transformed_query[key])
update_xml(top, wiki, "/content/drive/MyDrive/IRDM/wikified_query_test.xml")

ParseError: ignored


#This section converts files into indexes for Podcast Small Extended  

In [ ]:
indexer = pt.TRECCollectionIndexer("/content/drive/MyDrive/IRDM/xml_dir/sample_without_test",blocks=True,meta={'docno':50,'text': 2048,})

In [ ]:
indexref = indexer.index(['/content/drive/MyDrive/IRDM/just_doc_file_test.xml'])

23:43:06.520 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (5SiVsFevoaSwa9kx9EA5x6_2520) - further warnings are suppressed
23:43:49.151 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 5 empty documents


In [ ]:
wikified= pt.TRECCollectionIndexer("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raws_test",blocks=True,meta={'docno':50,'text': 2048,})
wikied= wikified.index(['/content/drive/MyDrive/IRDM/wikified_file_for_wiki_test.xml'])

In [ ]:
wiki2= pt.TRECCollectionIndexer("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_2_test",blocks=True,meta={'docno':50,'text': 2048,})
wikied= wiki2.index(['/content/drive/MyDrive/IRDM/wikified_file_for_wiki_freq_2_test.xml'])

In [ ]:
wiki5= pt.TRECCollectionIndexer("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_5_test",blocks=True,meta={'docno':50,'text': 2048,})
wikied= wiki5.index(['/content/drive/MyDrive/IRDM/wikified_file_for_wiki_freq_5_test.xml'])

In [ ]:
sample_set_we = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/sample_without_test/data.properties")
raw= pt.IndexFactory.of(sample_set_we)

In [ ]:
wiki_instance = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raws_test/data.properties")
wiki_no_difference= pt.IndexFactory.of(wiki_instance)

In [ ]:
wiki_instance_2 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_2_test/data.properties")
wiki_no_difference_2= pt.IndexFactory.of(wiki_instance_2)

In [ ]:
wiki_instance_5 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_5/data.properties")
wiki_no_difference_5= pt.IndexFactory.of(wiki_instance_5)

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_test_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


def write_trec(df, args):
    import sys
    print('query-id Q0 document-id rank score STANDARD')
    from contextlib import redirect_stdout

    with open('/content/drive/MyDrive/IRDM/output.txt', 'w') as f:
      with redirect_stdout(f):
        for t in df.itertuples():
          qid, rank, docno, score = t[1], t[2], t[3], t[4]

          episode, timestamp = docno.split('_')
          start_time = str(float(timestamp.split('-')[0]))
          episode_id = id_template.format(episode, start_time)
          orig_stdout = sys.stdout
        
          print('{} {} {} {} {}'.format(
              qid, '0', episode_id, rank, score,))




import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

In [ ]:
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.326443935878028,
 'ndcg': 0.570547109064818,
 'ndcg_cut_30': 0.3725153266746206,
 'P_10': 0.40833333333333327}

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})




import pandas as pd

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("Qe ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

14:26:49.228 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
Qe 
{'map': 0.3594896727893227, 'ndcg': 0.5818676337919272, 'ndcg_cut_30': 0.40024616726520473, 'P_10': 0.4729166666666667}


In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})




import pandas as pd

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("Qe ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

14:54:18.469 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
Qe 
{'map': 0.37359149332068187, 'ndcg': 0.5755390063437716, 'ndcg_cut_30': 0.3979215958035478, 'P_10': 0.47708333333333347}


In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


def write_trec(df, args):
    import sys
    print('query-id Q0 document-id rank score STANDARD')
    from contextlib import redirect_stdout

    with open('/content/drive/MyDrive/IRDM/output.txt', 'w') as f:
      with redirect_stdout(f):
        for t in df.itertuples():
          qid, rank, docno, score = t[1], t[2], t[3], t[4]

          episode, timestamp = docno.split('_')
          start_time = str(float(timestamp.split('-')[0]))
          episode_id = id_template.format(episode, start_time)
          orig_stdout = sys.stdout
        
          print('{} {} {} {} {}'.format(
              qid, '0', episode_id, rank, score,))




import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

In [ ]:
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.0, 'ndcg': 0.0, 'ndcg_cut_30': 0.0, 'P_10': 0.0}

{'map': 0.25422411493243313,
 'ndcg': 0.5108938918540078,
 'P_10': 0.30000000000000004}

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

In [ ]:
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.20256940525228698,
 'ndcg': 0.48670058356898993,
 'ndcg_cut_30': 0.2874713617454185,
 'P_10': 0.3125}

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_test_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


def get_pl(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "PL2"},metadata=["docno",'text'])

bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


pl2 = get_pl(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.22052363593952226,
 'ndcg': 0.5069845630459082,
 'ndcg_cut_30': 0.3045576680805741,
 'P_10': 0.36250000000000004}

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})

bm25 = get_retr(wiki_no_difference_2, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference_2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()




import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.20256940525228698,
 'ndcg': 0.48670058356898993,
 'ndcg_cut_30': 0.2874713617454185,
 'P_10': 0.3125}

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_test_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


def get_pl(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "PL2"},metadata=["docno",'text'])

def get_qe(index):
    return pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=20, fb_docs=20) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})




bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


pl2 = get_pl(wiki_no_difference_5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


qe = get_qe(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()    




import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= 0.20*qe+0.20*bm25+0.50*dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

NameError: ignored

In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker()
duoT5 = DuoT5ReRanker()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-kmseezio/python-terrier_72517e5100df461d8608c86a99a0cf34
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-kmseezio/python-terrier_72517e5100df461d8608c86a99a0cf34
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
  Attempting uninstall: pyjnius
    Found existing installation: pyjnius 1.3.0
    Uninstalling pyjnius-1.3.0:
      Successfully uninstalled pyjnius-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-terrier 0.8.1 requires pyjnius~=1.3.0, but you have pyjnius 1.4.2 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/terrierteam/pyterrier_t5.git to /tmp/pip-req-build-6i23iiro
  Running command git clone -q https://github.com/terrierteam/pyterrier_t5.git /tmp/pip-req-build-6i23iiro
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
  Using cached pyjnius-1.3.0-cp37-cp37m-manylinux2010_x86_64.whl (1.1 MB)
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 120 kB 58.9 MB/s 
  Created wheel for pyterrier-t5: filename=pyterrier_t5-0.0.1-py3-none-any.whl size=4880 sha256=ef5714020a20d588e5faf57a1ec54cec8049f4ae434bcd2be6cbf1ce9217db88
  Stored in directory: /tmp/pip-ephem-wheel-cache-bt2yvpe8/wheels/f7/03/ca/cf7d91992877b77796d943cfe581f27398f9fa6be41236f7f6
Successfully built pyterrier-t5
  Attempting uninstall: pyjnius
    Found existing installation: pyjnius 1.4.2
  

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
bm25 = pt.BatchRetrieve(wiki_no_difference, wmodel="BM25")
mono_pipeline = bm25 >> pt.text.get_text(wiki_no_difference,metadata=["docno","text"]) >> monoT5
duo_pipeline = mono_pipeline % 50 >> duoT5

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


def get_pl(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "PL2"},metadata=["docno",'text'])

def get_qe(index):
    return pt.BatchRetrieve(index, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=20, fb_docs=20) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "DPH"})




bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


pl2 = get_pl(wiki_no_difference_5) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


qe = get_qe(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()    




import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= qe+bm25+dph+ duo_pipeline
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

01:08:15.255 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1


monoT5:   0%|          | 0/2000 [00:00<?, ?batches/s]

duoT5:   0%|          | 0/8 [00:00<?, ?queries/s]

{'map': 0.22172364202402955,
 'ndcg': 0.5094609680367183,
 'ndcg_cut_30': 0.32367161580473314,
 'P_10': 0.3625}